<a href="https://colab.research.google.com/github/chasdabigone/RoD_Equipment_Search/blob/main/rod_ascii_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 1. Upload your file to the gui on the left as 'image.jpg'
# 2. Click runtime->run all
# 3. picture.html (viewable with browser) and picture.txt (ascii) will be output as files
# 4. adjust columns slider to change width of picture (must re-run program)

In [8]:
pip install ascii_magic

In [19]:
from ascii_magic import AsciiArt, Back
from PIL import ImageEnhance
import re

# Change columns to adjust width of picture. 79 is max
columns = 79 #@param {type: "slider", min: 20, max: 79}
brightness = 1.1 #@param {type: "slider", min: 0.1, max: 2, step: 0.1}
contrast = 1.1 #@param {type: "slider", min: 0.1, max: 2, step: 0.1}


def rgbto256(r, g, b):
    # XTerm Color Number = 16 + 36 * R + 6 * G + B | 0 <= R,G,B <= 5
    r = int(r)
    g = int(g)
    b = int(b)
    return 16 + 36 * (r // 51) + 6 * (g // 51) + (b // 51)

def hextorgb(hex_color):
    # Extract RGB values from a hex color code
    hex_color = hex_color.lstrip('#')  # Remove leading '#' if present
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    return r, g, b

colors_to_rod = {'0': '&x', '1': '&r', '2': '&g', '3': '&O', '4': '&b', '5': '&p', '6': '&c', '7': '&w', '8': '&z', '9': '&R', '10': '&G', '11': '&Y', '12': '&B', '13': '&P', '14': '&C', '15': '&W', '16': '&e000', '40': '&e040', '64': '&e120', '88': '&e200', '112': '&e240', '136': '&e320', '160': '&e400', '184': '&e440', '208': '&e520', '200': '&e504', '17': '&e001', '41': '&e041', '65': '&e121', '89': '&e201', '113': '&e241', '137': '&e321', '161': '&e401', '185': '&e441', '209': '&e521', '201': '&e505', '18': '&e002', '42': '&e042', '66': '&e122', '90': '&e202', '114': '&e242', '138': '&e322', '162': '&e402', '186': '&e442', '210': '&e910', '202': '&e510', '19': '&e003', '43': '&e043', '67': '&e123', '91': '&e203', '115': '&e243', '139': '&e323', '163': '&e403', '187': '&e443', '211': '&e911', '203': '&e511', '20': '&e004', '44': '&e044', '68': '&e124', '92': '&e204', '116': '&e244', '140': '&e324', '164': '&e404', '188': '&e444', '212': '&e912', '204': '&e512', '21': '&e005', '45': '&e045', '69': '&e125', '93': '&e205', '117': '&e245', '141': '&e325', '165': '&e405', '189': '&e445', '213': '&e913', '205': '&e513', '22': '&e010', '46': '&e050', '70': '&e130', '94': '&e210', '118': '&e250', '142': '&e330', '166': '&e410', '190': '&e450', '214': '&e914', '23': '&e011', '47': '&e051', '71': '&e131', '95': '&e211', '119': '&e251', '143': '&e331', '167': '&e411', '191': '&e451', '215': '&e915', '24': '&e012', '48': '&e052', '72': '&e132', '96': '&e212', '120': '&e252', '144': '&e332', '168': '&e412', '192': '&e452', '216': '&e532', '25': '&e013', '49': '&e053', '73': '&e133', '97': '&e213', '121': '&e253', '145': '&e333', '169': '&e413', '193': '&e453', '217': '&e533', '26': '&e014', '50': '&e054', '74': '&e134', '98': '&e214', '122': '&e254', '146': '&e334', '170': '&e414', '194': '&e454', '218': '&e534', '27': '&e015', '51': '&e055', '75': '&e135', '99': '&e215', '123': '&e255', '147': '&e335', '171': '&e415', '195': '&e455', '219': '&e535', '28': '&e020', '52': '&e100', '76': '&e140', '100': '&e220', '124': '&e300', '148': '&e340', '172': '&e420', '196': '&e500', '220': '&e920', '240': '&e940', '29': '&e021', '53': '&e101', '77': '&e141', '101': '&e221', '125': '&e301', '149': '&e341', '173': '&e421', '197': '&e501', '221': '&e921', '241': '&e941', '30': '&e022', '54': '&e102', '78': '&e142', '102': '&e222', '126': '&e302', '150': '&e342', '174': '&e422', '198': '&e502', '222': '&e922', '242': '&e942', '31': '&e023', '55': '&e103', '79': '&e143', '103': '&e223', '127': '&e303', '151': '&e343', '175': '&e423', '199': '&e503', '223': '&e923', '243': '&e943', '32': '&e024', '56': '&e104', '80': '&e144', '104': '&e224', '128': '&e304', '152': '&e344', '176': '&e424', '224': '&e924', '244': '&e944', '33': '&e025', '57': '&e105', '81': '&e145', '105': '&e225', '129': '&e305', '153': '&e345', '177': '&e425', '225': '&e925', '245': '&e945', '34': '&e030', '58': '&e110', '82': '&e150', '106': '&e230', '130': '&e310', '154': '&e350', '178': '&e430', '226': '&e550', '230': '&e554', '250': '&e950', '35': '&e031', '59': '&e111', '83': '&e151', '107': '&e231', '131': '&e311', '155': '&e351', '179': '&e431', '227': '&e551', '231': '&e555', '251': '&e951', '36': '&e032', '60': '&e112', '84': '&e152', '108': '&e232', '132': '&e312', '156': '&e352', '180': '&e432', '228': '&e552', '232': '&e932', '252': '&e952', '37': '&e033', '61': '&e113', '85': '&e153', '109': '&e233', '133': '&e313', '157': '&e353', '181': '&e433', '229': '&e553', '233': '&e933', '253': '&e953', '38': '&e034', '62': '&e114', '86': '&e154', '110': '&e234', '134': '&e314', '158': '&e354', '182': '&e434', '206': '&e514', '234': '&e934', '254': '&e954', '39': '&e035', '63': '&e115', '87': '&e155', '111': '&e235', '135': '&e315', '159': '&e355', '183': '&e435', '207': '&e515', '235': '&e935', '255': '&e955'}

widthRatio = 2.2
# create html file
def create():
  global widthRatio
  my_art = AsciiArt.from_image('/content/image.jpg')
  my_art.image = ImageEnhance.Brightness(my_art.image).enhance(brightness)
  my_art.image = ImageEnhance.Contrast(my_art.image).enhance(contrast)
  my_art.to_html_file('/content/picture.html',columns=columns,width_ratio=widthRatio)
  my_art.to_file('/content/picture.txt',columns=columns,width_ratio=widthRatio)

  with open(r"/content/picture.txt", 'r') as fp:
      x = len(fp.readlines())
      if x != 24:
        widthRatio = widthRatio * x / 24
        create()

create()

# Read the HTML file
with open("/content/picture.html", "r", encoding="utf-8") as html_file:
    html_content = html_file.read()

# Define a regular expression pattern to match the style and text within <span> elements
span_pattern = r'<span style="color: #(.*?)">(.*?)<\/span>'

# Extract the pre element containing the styled text
pre_match = re.search(r'<pre.*?>(.*?)</pre>', html_content, re.DOTALL)

if pre_match:
    pre_content = pre_match.group(1)

    # Find all <span> elements and extract text and color information
    span_matches = re.findall(span_pattern, pre_content, re.DOTALL)

    # Create a new text with color codes and adding a line break every 80 characters
    text_with_color = ''
    character_count = 0
    current_color = None

    for color, text in span_matches:
      # convert the colors from hex to rgb, then rgb to 256, then finally 256 to RoD
        color = hextorgb(color)
        color = rgbto256(*color)
        color = colors_to_rod.get(str(color), color)

        # add line break if column number exceeded
        if character_count >= columns:
            text_with_color += '\n'
            character_count = 0
            current_color = None  # Reset the current color when starting a new line

        if color != current_color:
            # If the color has changed, add it to the text
            text_with_color += color
            current_color = color


        text_with_color += text
        character_count += len(text)

    # Write the text with color codes to a text file
    with open('picture.txt', 'w') as output_file:
        output_file.write(text_with_color)

    print('Conversion complete. The output has been saved to "picture.txt."')

else:
    print('No <pre> element found in the HTML content.')




Conversion complete. The output has been saved to "picture.txt."
